In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import plotly
import plotly.graph_objects as go


from astral import LocationInfo
from astral.sun import sun


In [2]:
from custom_utils import *

In [3]:
def get_raw_data():
    data_path = "./../data/"
    file_name = "data.csv"
    data = pd.read_csv(data_path + file_name)
    data = first_preprocess(data, data_path)
    return data

In [4]:
data1 = pd.read_csv("sh_pred.csv")
data2 = pd.read_csv("others_pred.csv")

In [5]:
data1.head()

,time,sh_predictions
0,2021-01-01 02:00:00+00:00,3.706257
1,2021-01-01 03:00:00+00:00,4.726034
2,2021-01-01 04:00:00+00:00,5.173534
3,2021-01-01 05:00:00+00:00,4.959734
4,2021-01-01 06:00:00+00:00,4.712812


In [6]:
data2.head()

,time,others_predictions
0,2021-01-01 02:00:00+00:00,0.202141
1,2021-01-01 03:00:00+00:00,0.216787
2,2021-01-01 04:00:00+00:00,0.219529
3,2021-01-01 05:00:00+00:00,0.236178
4,2021-01-01 06:00:00+00:00,0.236935


In [7]:
building_data = data1.merge(data2, on="time", how="inner")
building_data.head()

,time,sh_predictions,others_predictions
0,2021-01-01 02:00:00+00:00,3.706257,0.202141
1,2021-01-01 03:00:00+00:00,4.726034,0.216787
2,2021-01-01 04:00:00+00:00,5.173534,0.219529
3,2021-01-01 05:00:00+00:00,4.959734,0.236178
4,2021-01-01 06:00:00+00:00,4.712812,0.236935


In [8]:
building_data["time"] = pd.to_datetime(building_data["time"], format="mixed")

In [9]:
raw_data = get_raw_data()
raw_data.columns

d:\RA_work\timeseries_data\notebooks\custom_utils.py:85: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data[column] = data[column].interpolate(method="linear", limit_direction="both")


Index(['space_heating', 'hot_water', 'sockets', 'lighting', 'bld_engcons',
       'car_chargers', 'weekend', 'bank_holiday', 'day_of_week',
       'day_of_month', 'month', 'year', 'forecast_temperature',
       'forecast_feelslike', 'forecast_weathertype', 'forecast_windspeed',
       'forecast_uvindex', 'forecast_precipitationprobability',
       'forecast_visibility', 'week_of_year', 'sine_hour', 'cos_hour',
       'sine_forecast_winddirection', 'cos_forecast_winddirection'],
      dtype='object')

In [10]:
exo_columns = ['forecast_temperature', 
               #'forecast_feelslike', 
               # 'forecast_weathertype',
       # 'forecast_windspeed', 'forecast_uvindex',
       # 'forecast_precipitationprobability', 'forecast_visibility',
       # 'week_of_year', 'daylight_hours', 'is_daylight', 
       'sine_hour', 'cos_hour', 
       # 'sine_month', 'cos_month', 'sine_day_of_week',
       # 'cos_day_of_week', 'sine_forecast_winddirection',
       # 'cos_forecast_winddirection', 'sine_sunrise_hour', 'cos_sunrise_hour',
       # 'sine_sunset_hour', 'cos_sunset_hour']
]
endo_columns = ['weekend', "month", "day_of_week"] # 'bank_holiday']

subset_data = raw_data[exo_columns + endo_columns + ["bld_engcons"]]
subset_data.head()

,forecast_temperature,sine_hour,cos_hour,weekend,month,day_of_week,bld_engcons
datetime,,,,,,,
2021-01-01 00:00:00+00:00,3.0,0.000000,1.000000,0,1,5,3.629499
2021-01-01 01:00:00+00:00,3.0,0.258819,0.965926,0,1,5,4.215506
2021-01-01 02:00:00+00:00,3.0,0.500000,0.866025,0,1,5,5.994066
2021-01-01 03:00:00+00:00,2.0,0.707107,0.707107,0,1,5,5.954252
2021-01-01 04:00:00+00:00,2.0,0.866025,0.500000,0,1,5,5.620296


In [11]:
subset_data.index.names = ['time']
subset_data.head()

,forecast_temperature,sine_hour,cos_hour,weekend,month,day_of_week,bld_engcons
time,,,,,,,
2021-01-01 00:00:00+00:00,3.0,0.000000,1.000000,0,1,5,3.629499
2021-01-01 01:00:00+00:00,3.0,0.258819,0.965926,0,1,5,4.215506
2021-01-01 02:00:00+00:00,3.0,0.500000,0.866025,0,1,5,5.994066
2021-01-01 03:00:00+00:00,2.0,0.707107,0.707107,0,1,5,5.954252
2021-01-01 04:00:00+00:00,2.0,0.866025,0.500000,0,1,5,5.620296


In [12]:
tmp1 = subset_data.reset_index(drop=0)
tmp2 = building_data.copy()

In [13]:
building_data = tmp1.merge(tmp2, on="time", how="left")
building_data.isna().sum()

time                    0
forecast_temperature    0
sine_hour               0
cos_hour                0
weekend                 0
month                   0
day_of_week             0
bld_engcons             0
sh_predictions          3
others_predictions      3
dtype: int64

In [14]:
building_data["sh_predictions"] = building_data["sh_predictions"].interpolate(limit_direction="both", method="linear")
building_data["others_predictions"] = building_data["others_predictions"].interpolate(limit_direction="both", method="linear")

In [15]:
building_data = building_data.set_index("time")
building_data = building_data.asfreq("h")

In [16]:
building_data.head()

,forecast_temperature,sine_hour,cos_hour,weekend,month,day_of_week,bld_engcons,sh_predictions,others_predictions
time,,,,,,,,,
2021-01-01 00:00:00+00:00,3.0,0.000000,1.000000,0,1,5,3.629499,3.706257,0.202141
2021-01-01 01:00:00+00:00,3.0,0.258819,0.965926,0,1,5,4.215506,3.706257,0.202141
2021-01-01 02:00:00+00:00,3.0,0.500000,0.866025,0,1,5,5.994066,3.706257,0.202141
2021-01-01 03:00:00+00:00,2.0,0.707107,0.707107,0,1,5,5.954252,4.726034,0.216787
2021-01-01 04:00:00+00:00,2.0,0.866025,0.500000,0,1,5,5.620296,5.173534,0.219529


In [17]:
bldg_columns = exo_columns + endo_columns + ["sh_predictions", "others_predictions"] + ["bld_engcons"]

# bld_data_scaler, bld_transformed_data = create_std_scaler(building_data, bldg_columns[:-1])
# bld_target_scaler, bld_transformed_target_data = create_std_scaler(building_data, bldg_columns[-1])

# bld_transformed_data = pd.merge(
#     bld_transformed_data, bld_transformed_target_data,
#     left_index=True,
#     right_index=True
# )

building_train_data = building_data[building_data.index < "2023-01-01"]
building_test_data = building_data[building_data.index >= "2023-01-01"]                            

In [18]:
building_train_data.head()

,forecast_temperature,sine_hour,cos_hour,weekend,month,day_of_week,bld_engcons,sh_predictions,others_predictions
time,,,,,,,,,
2021-01-01 00:00:00+00:00,3.0,0.000000,1.000000,0,1,5,3.629499,3.706257,0.202141
2021-01-01 01:00:00+00:00,3.0,0.258819,0.965926,0,1,5,4.215506,3.706257,0.202141
2021-01-01 02:00:00+00:00,3.0,0.500000,0.866025,0,1,5,5.994066,3.706257,0.202141
2021-01-01 03:00:00+00:00,2.0,0.707107,0.707107,0,1,5,5.954252,4.726034,0.216787
2021-01-01 04:00:00+00:00,2.0,0.866025,0.500000,0,1,5,5.620296,5.173534,0.219529


In [19]:
# bldg_forecaster = ForecasterAutoreg(regressor=LinearRegression(n_jobs=-1), lags=24)
bldg_forecaster = ForecasterAutoreg(
    # regressor=RandomForestRegressor(criterion="squared_error", 
    #                                 n_estimators=750,
    #                                 max_depth=18,
    #                                 n_jobs=-1), 
    regressor=LinearRegression(), lags=2)
bldg_forecaster.fit(
    y=building_train_data['bld_engcons'],
    exog=building_train_data[bldg_columns[:-1]]
)

#### building forecaster training data

In [20]:
def custom_metric(y_true, y_pred):
    '''
    Calculate the mean squared error using only the predicted values of the last
    3 months of the year.
    '''
    return calculate_smape(actual=y_true, predicted=y_pred)

In [21]:
metric, bldg_predictions_training = backtesting_forecaster(
    forecaster         = bldg_forecaster,
    y                  = building_train_data['bld_engcons'],
    exog               = building_train_data[bldg_columns[:-1]],
    steps              = 1,
    metric             = custom_metric,
    initial_train_size = None,
    refit              = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

print(f"Backtest error: {metric:.2f}")
bldg_predictions_training.head()

  0%|          | 0/17518 [00:00<?, ?it/s]

Backtest error: 40.78


,pred
2021-01-01 02:00:00+00:00,4.035089
2021-01-01 03:00:00+00:00,5.115278
2021-01-01 04:00:00+00:00,5.551019
2021-01-01 05:00:00+00:00,5.367221
2021-01-01 06:00:00+00:00,5.160618


In [22]:
bldg_forecaster

ForecasterAutoreg 
Regressor: LinearRegression() 
Lags: [1 2] 
Transformer for y: None 
Transformer for exog: None 
Window size: 2 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['forecast_temperature', 'sine_hour', 'cos_hour', 'weekend', 'month', 'day_of_week', 'sh_predictions', 'others_predictions'] 
Training range: [Timestamp('2021-01-01 00:00:00+0000', tz='UTC'), Timestamp('2022-12-31 23:00:00+0000', tz='UTC')] 
Training index type: DatetimeIndex 
Training index frequency: h 
Regressor parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2024-06-19 05:07:56 
Last fit date: 2024-06-19 05:07:56 
Skforecast version: 0.12.1 
Python version: 3.11.8 
Forecaster id: None 

In [23]:
bldg_predictions_training["y"] = building_train_data["bld_engcons"]
bldg_predictions_training = bldg_predictions_training.reset_index(drop=0)
# bldg_predictions_training[["pred", "y"]] = bld_target_scaler.inverse_transform(bldg_predictions_training[["pred", "y"]])
bldg_predictions_training = bldg_predictions_training.rename(columns={"index": "time"})

bldg_predictions_training.head()

,time,pred,y
0,2021-01-01 02:00:00+00:00,4.035089,5.994066
1,2021-01-01 03:00:00+00:00,5.115278,5.954252
2,2021-01-01 04:00:00+00:00,5.551019,5.620296
3,2021-01-01 05:00:00+00:00,5.367221,5.750796
4,2021-01-01 06:00:00+00:00,5.160618,5.704551


In [24]:
def plot_predictions(plot_data, title_text):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["y"].to_numpy(),
                        mode='lines+markers',
                        name='actual'))
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["pred"].to_numpy(),
                        mode='lines+markers',
                        name='predictions'))
    fig.update_layout(title_text=title_text)
    fig.show()

In [25]:
plot_predictions(bldg_predictions_training, title_text="Building engergy consumption Training data")

#### building forecaster testing data with refit

In [26]:
metric, bldg_predictions = backtesting_forecaster(
    forecaster         = bldg_forecaster,
    y                  = building_data['bld_engcons'],
    exog               = building_data[bldg_columns[:-1]],
    steps              = 1,
    metric             = custom_metric,
    initial_train_size = building_data[building_data.index < "2023-01-01"].shape[0],
    refit              = 1000, # total refit 3 + 3 
    fixed_train_size   = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

print(f"Backtest error: {metric:.2f}")
bldg_predictions.head()

  0%|          | 0/8760 [00:00<?, ?it/s]

Backtest error: 29.74


,pred
2023-01-01 00:00:00+00:00,1.841971
2023-01-01 01:00:00+00:00,3.123192
2023-01-01 02:00:00+00:00,3.169991
2023-01-01 03:00:00+00:00,2.219139
2023-01-01 04:00:00+00:00,2.050785


In [27]:
bldg_predictions.head()

,pred
2023-01-01 00:00:00+00:00,1.841971
2023-01-01 01:00:00+00:00,3.123192
2023-01-01 02:00:00+00:00,3.169991
2023-01-01 03:00:00+00:00,2.219139
2023-01-01 04:00:00+00:00,2.050785


In [28]:
bldg_predictions["y"] = building_test_data["bld_engcons"]
bldg_predictions = bldg_predictions.reset_index(drop=0)
# bldg_predictions[["pred", "y"]] = bld_target_scaler.inverse_transform(bldg_predictions[["pred", "y"]])
bldg_predictions = bldg_predictions.rename(columns={"index": "time"})
bldg_predictions.head()

plot_predictions(bldg_predictions, title_text="Building engergy consumption Testing data")

#### Metrics

In [29]:
cal_metrics(bldg_predictions_training["pred"].to_numpy(), bldg_predictions_training["y"].to_numpy())

,MAE,MAPE,R2_Score,SMAPE,nRMSE,RMSE,MASE
0,0.439862,45.0,0.621647,40.78,0.076175,0.698357,-0.44


In [30]:
cal_metrics(bldg_predictions["pred"].to_numpy(), bldg_predictions["y"].to_numpy())

,MAE,MAPE,R2_Score,SMAPE,nRMSE,RMSE,MASE
0,0.466361,33.3,0.563897,29.74,0.098087,0.718351,-0.47


#### Explore Error / Residual

In [ ]:
assert (bldg_predictions[bldg_predictions["pred"] <= 0].shape[0] > 0) == False

In [ ]:
q1_2023 = bldg_predictions[(bldg_predictions["time"] >= "2023-01-01") & (bldg_predictions["time"] < "2023-04-01")]  # jan - fev - march 
q2_2023 = bldg_predictions[(bldg_predictions["time"] >= "2023-04-01") & (bldg_predictions["time"] < "2023-07-01")] # april - may - june
q3_2023 = bldg_predictions[(bldg_predictions["time"] >= "2023-07-01") & (bldg_predictions["time"] < "2023-10-01")] # july - aug - sept
q4_2023 = bldg_predictions[(bldg_predictions["time"] >= "2023-10-01") & (bldg_predictions["time"] <= "2023-12-31")] # oct - nov - dec

q1_2023.shape, q2_2023.shape, q3_2023.shape, q4_2023.shape 

In [ ]:
q1_2022 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2022-01-01") & (bldg_predictions_training["time"] < "2022-04-01")]  # jan - fev - march 
q2_2022 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2022-04-01") & (bldg_predictions_training["time"] < "2022-07-01")] # april - may - june
q3_2022 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2022-07-01") & (bldg_predictions_training["time"] < "2022-10-01")] # july - aug - sept
q4_2022 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2022-10-01") & (bldg_predictions_training["time"] <= "2022-12-31")] # oct - nov - dec

q1_2022.shape, q2_2022.shape, q3_2022.shape, q4_2022.shape 

In [ ]:
q1_2021 = bldg_predictions_training[bldg_predictions_training["time"] < "2021-04-01"]  # jan - fev - march 
q2_2021 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2021-04-01") & (bldg_predictions_training["time"] < "2021-07-01")] # april - may - june
q3_2021 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2021-07-01") & (bldg_predictions_training["time"] < "2021-10-01")] # july - aug - sept
q4_2021 = bldg_predictions_training[(bldg_predictions_training["time"] >= "2021-10-01") & (bldg_predictions_training["time"] <= "2021-12-31")] # oct - nov - dec

q1_2021.shape, q2_2021.shape, q3_2021.shape, q4_2021.shape 

In [ ]:
q_df = pd.DataFrame({})

def print_metrics(q_list, year):
    global q_df
    for idx, quar in enumerate(q_list):
        m_df = cal_metrics(quar["pred"].to_numpy(), quar["y"].to_numpy())
        m_df["year"] = year
        m_df["quarter"] = idx +1
        q_df = pd.concat([q_df, m_df], axis=0).reset_index(drop=1)

In [ ]:
print_metrics([q1_2023, q2_2023, q3_2023, q4_2023], "2023")
print_metrics([q1_2022, q2_2022, q3_2022, q4_2022], "2022")
print_metrics([q1_2021, q2_2021, q3_2021, q4_2021], "2021")

In [ ]:
q_df

In [ ]:
sns.catplot(data=q_df, x='year', y='SMAPE', hue='quarter', kind='bar', palette="pastel")

In [ ]:
def _plot_dist(quar, year, idx):
    # distribution plot
    ax = sns.displot(data=quar["error"])
    plt.title(f"Residual distribution for {year}: Q{idx + 1}")
    plt.tight_layout()
    plt.savefig(f".././assets/error_plots/{year}_Q{idx + 1}.jpeg", format="jpeg")
    plt.show()


def _plot_box(quar,  year, idx):
    # error plot
    ax = sns.boxplot(data=quar[["error"]])
    plt.title(f"Outliers for {year}: Q{idx + 1}")
    plt.tight_layout()
    plt.savefig(f".././assets/error_plots/{year}_Q{idx + 1}_outlier.jpeg", format="jpeg")
    plt.show()

def plot_dist(q_list, year):
    for idx, quar in enumerate(q_list):
        quar.loc[:, "error"] = quar["pred"] - quar["y"]

        _plot_dist(quar, year, idx)

        _plot_box(quar, year, idx)

In [ ]:
plot_dist([q1_2021, q2_2021, q3_2021, q4_2021], "2021")
plot_dist([q1_2022, q2_2022, q3_2022, q4_2022], "2022")
plot_dist([q1_2023, q2_2023, q3_2023, q4_2023], "2023")

##### Apply cutoffs from error analysis and outlier analysis

In [ ]:
data_mapping = {
    "2021": {"data": [q1_2021, q2_2021, q3_2021, q4_2021], 
             "error_cutoffs": [(-0.2, 0.2), (-0.1, 0.1), (-0.1,0.1), (-0.2, 0.2)]},
    "2022": {"data": [q1_2022, q2_2022, q3_2022, q4_2022], 
             "error_cutoffs": [(-0.2, 0.2), (-0.1, 0.1), (-0.1, 0.1), (-0.2, 0.2)]},
    "2023": {"data": [q1_2023, q2_2023, q3_2023, q4_2023], 
             "error_cutoffs": [(-2, 2), (-1, 1), (-0.75, 0.75), (-1.75, 1.75)]}
}

In [ ]:
def apply_error_cutoffs(q_list, error_cutoffs):
    subset_qlist = []
    for quar, cutoff in zip(q_list, error_cutoffs):
        quar.loc[:, "error"] = quar['pred'] - quar["y"]
        quar = quar[(quar["error"] >= cutoff[0]) & (quar["error"] <= cutoff[1])]
        subset_qlist.append(quar)
    return subset_qlist

In [ ]:
q_df = pd.DataFrame({})
for year in data_mapping:
    data = apply_error_cutoffs(data_mapping[year]["data"], data_mapping[year]["error_cutoffs"])
    print_metrics(data, year)

In [ ]:
q_df

In [ ]:
sns.catplot(data=q_df, x='year', y='SMAPE', hue='quarter', kind='bar', palette="pastel")

In [ ]:
sns.catplot(data=q_df, x='year', y='nRMSE', hue='quarter', kind='bar', palette="pastel")

In [ ]:
sns.catplot(data=q_df, x='year', y='RMSE', hue='quarter', kind='bar', palette="pastel")

#### Explore prediction intervals

In [ ]:
bldg_forecaster

In [ ]:
prob_intervals = bldg_forecaster.predict_interval(
    steps    = building_test_data.shape[0],
    exog = building_test_data[bldg_columns[:-1]],
    interval = [5, 95],
    n_boot   = 200
)

In [ ]:
# prob_intervals[['pred', "lower_bound", "upper_bound"]] = bld_target_scaler.inverse_transform(prob_intervals[["pred", "lower_bound", "upper_bound"]])

In [ ]:
# prob_intervals[["actual"]] = bld_target_scaler.inverse_transform(building_test_data[["bld_engcons"]])
# prob_intervals.head()

In [ ]:
plot_data = prob_intervals.reset_index(drop=0)
plot_data = plot_data.rename(columns={"index": "time"})
plot_data.head()

In [ ]:
def plot_predictions_intervals(plot_data, title_text):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["actual"].to_numpy(),
                        mode='lines+markers',
                        name='actual',
                        marker=dict(color="green")))
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["pred"].to_numpy(),
                        mode='lines+markers',
                        name='predictions',
                        marker=dict(color="blue")))
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["lower_bound"].to_numpy(),
                        mode='lines',
                        marker=dict(color="red"),
                        line=dict(width=0),
                        showlegend=False,
                        name="lower bound",
                       fillcolor='rgba(240, 67, 176, 0.3)',
                       fill='tonexty'))
    fig.add_trace(go.Scatter(x=plot_data["time"].to_numpy(), y=plot_data["upper_bound"].to_numpy(),
                        mode='lines',
                        marker=dict(color="red"),
                        line=dict(width=0),
                        showlegend=False,
                        name="upper bound", 
                        fillcolor='rgba(240, 67, 176, 0.3)',
                       fill='tonexty'))
    fig.update_layout(
        yaxis_title="Energy Consumption in KWh",
        title_text=title_text, 
        hovermode="x"
    )
    fig.show()
    asset_path = '.././assets/'
    file_name = 'composite_model_4b.html'
    plotly.offline.plot(fig, filename=asset_path + file_name)

In [ ]:
plot_predictions_intervals(plot_data, "Building Energy Conumption Prediction Year 2023")